In [1]:
consumer_PATH = "consumer_complaints.csv"
TEXT_DATA_DIR = consumer_PATH
MAX_SEQUENCE_LENGTH = 500
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.4

In [2]:
!pip install tensorflow==1.15.2

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install seaborn

You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import re
import itertools
import string
from io import BytesIO
from collections import Counter
from time import time
import datetime
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.utils import to_categorical

In [5]:
# from google.colab import files
# uploaded = files.upload()

### Data Preprocessing

In [6]:

def clean_str(string):
    """
    Tokenization/string cleaning (partially modified)
    """
    string = re.sub(r"[^A-Za-z0-9()!?\'\`%$]", " ", string) # keep also %$ but removed comma
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\$", " $ ", string) #yes, isolate $
    string = re.sub(r"\%", " % ", string) #yes, isolate %
    string = re.sub(r"\s{2,}", " ", string)
    
    # fixing XXX and xxx like as word
    string = re.sub(r'\S*(x{2,}|X{2,})\S*',"xxx",string)
    # removing non ascii
    string = re.sub(r'[^\x00-\x7F]+', "", string) 
    
    return string.strip().lower()
def get_complaints(path):
    
    # Read the input dataset 
    d = pd.read_csv(path, 
                    usecols=('product','consumer_complaint_narrative'),
                    dtype={'consumer_complaint_narrative': object})
    # Only interested in data with consumer complaints
    d=d[d['consumer_complaint_narrative'].notnull()]
    d=d[d['product'].notnull()]
    d.reset_index(drop=True,inplace=True)
    d['consumer_complaint_narrative'] = d['consumer_complaint_narrative'].apply(clean_str)
    
    le = preprocessing.LabelEncoder()
    label_name = d['product'].tolist()
    le.fit(label_name)
    d['label'] = le.transform(label_name)
    return d['consumer_complaint_narrative'].tolist(), d['product'].tolist(), d['label'].tolist()

### Naive Bayes Classification 

In [7]:
df =pd.read_csv(TEXT_DATA_DIR, 
                    usecols=('product','consumer_complaint_narrative'),
                    dtype={'consumer_complaint_narrative': object})

df = df[pd.notnull(df['consumer_complaint_narrative'])]

print(df.info())

col = ['product', 'consumer_complaint_narrative']
df = df[col]

print (df.columns)

df.columns = ['product', 'consumer_complaint_narrative']
########### Model #############
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(df['consumer_complaint_narrative'], df['product'], random_state = 0)

######## Count Vectorizer Classifier ################
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
clf = MultinomialNB().fit(X_train_counts, y_train)

print(clf.predict(count_vect.transform(["This company refuses to provide me verification and validation of debt per my right under the FDCPA. I do not believe this debt is mine."])))
print(clf.predict(count_vect.transform(["I am disputing the inaccurate information the Chex-Systems has on my credit report. I initially submitted a police report on XXXX/XXXX/16 and Chex Systems only deleted the items that I mentioned in the letter and not all the items that were actually listed on the police report. In other words they wanted me to say word for word to them what items were fraudulent. The total disregard of the police report and what accounts that it states that are fraudulent. If they just had paid a little closer attention to the police report I would not been in this position now and they would n't have to research once again. I would like the reported information to be removed : XXXX XXXX XXXX"])))

X_test_counts = count_vect.transform(X_test)
y_pred = clf.predict(X_test_counts)


# print(metrics.classification_report(y_test, y_pred, target_names=df['product'].unique()))
print(accuracy_score(y_test,y_pred))




######## TFIDF Classifier ################
# tfidf_vect = TfidfVectorizer(binary=True, use_idf=True)
# X_train_tfidf = tfidf_vect.fit_transform(X_train)

# clf1 = MultinomialNB().fit(X_train_tfidf, y_train)
# X_test_tfidf = tfidf_vect.transform(X_test)


# y_pred1 = clf1.predict(X_test_tfidf)
# print(metrics.classification_report(y_test, y_pred, target_names=df['product'].unique()))
# print(accuracy_score(y_test,y_pred1))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66806 entries, 190126 to 553096
Data columns (total 2 columns):
product                         66806 non-null object
consumer_complaint_narrative    66806 non-null object
dtypes: object(2)
memory usage: 1.5+ MB
None
Index(['product', 'consumer_complaint_narrative'], dtype='object')
['Debt collection']
['Credit reporting']
0.8068494791042989


In [8]:
naive_bayes_accuracy = accuracy_score(y_test,y_pred)
print("Naive Bayes Accuracy: {}".format(naive_bayes_accuracy))

Naive Bayes Accuracy: 0.8068494791042989


In [9]:
print(clf.predict(count_vect.transform(["I love my house but I can't pay it stop calling me"])))

['Debt collection']


In [10]:
print(clf.predict(count_vect.transform(["I love my house"])))

['Mortgage']


In [11]:
print(clf.predict(count_vect.transform(["My credit score is trash"])))

['Credit reporting']


In [12]:
y_train

311905           Credit reporting
272576    Bank account or service
275539            Debt collection
224667                   Mortgage
244070    Bank account or service
                   ...           
310182                   Mortgage
259720                   Mortgage
319684            Debt collection
311762                Credit card
314175    Bank account or service
Name: product, Length: 50104, dtype: object

### Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
# # X_train_counts = count_vect.fit_transform(X_train)
# # clf = MultinomialNB().fit(X_train_counts, y_train)

clf1 =  LogisticRegression()
clf1.fit(X_train_counts, y_train)

print(clf1.predict(count_vect.transform(["This company refuses to provide me verification and validation of debt per my right under the FDCPA. I do not believe this debt is mine."])))


# # pred = clf.predict(X_test)
# # experiment_result['LR'] = metrics.accuracy_score(np.argmax(y_val, axis=1), pred)


['Debt collection']


/Users/armandoordoricadelatorre/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [14]:
print(clf1.predict(count_vect.transform(["This company refuses to provide me verification and validation of debt per my right under the FDCPA. I do not believe this debt is mine."])))

['Debt collection']


In [15]:
print(clf1.predict(count_vect.transform(["I love my house"])))

['Debt collection']


In [16]:
X_test_counts_logreg = count_vect.transform(X_test)
y_pred = clf1.predict(X_test_counts_logreg)

# print(metrics.classification_report(y_test, y_pred, target_names=df['product'].unique()))
logistic_regression_accuracy = accuracy_score(y_test,y_pred)
print("Logistic Regression Accuracy: {}".format(logistic_regression_accuracy))

Logistic Regression Accuracy: 0.830738833672614


In [17]:
print(clf1.predict(count_vect.transform(["House loan"])))

['Mortgage']


### Summary

In [21]:
logistic_regression_accuracy

0.830738833672614

In [22]:
naive_bayes_accuracy

0.8068494791042989